# ĐỒ ÁN THỰC HÀNH
# CSC17104 – LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU

Giảng viên: Bùi Tiến Lên

Thông tin thành viên:
1. Nguyễn Văn Tuấn Đạt 19120472
2. Phan Xuân Hoài 20120481
3. 
4. 

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)

C:\Users\Dat\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


## A. Thu thập dữ liệu
- Tên dữ liệu: HR Analytics: Job Change of Data Scientists
- Nguồn lấy dữ liệu: kaggle
- Link dữ liệu: https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists
- License: CC0: Public Domain
- Mô tả về dữ liệu: dữ liệu gồm tập train và tập test. Trong đồ án này, nhóm sẽ sử dụng tập train.
            + Dữ liệu gồm 14 cột và 19158 dòng.

In [2]:
df = pd.read_csv('aug_train.csv', encoding='utf-8')
df.shape

(19158, 14)

In [3]:
df.head(10)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
5,21651,city_176,0.764,NaN,Has relevent experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0
6,28806,city_160,0.920,Male,Has relevent experience,no_enrollment,High School,NaN,5,50-99,Funded Startup,1,24,0.0
7,402,city_46,0.762,Male,Has relevent experience,no_enrollment,Graduate,STEM,13,<10,Pvt Ltd,>4,18,1.0
8,27107,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,7,50-99,Pvt Ltd,1,46,1.0
9,699,city_103,0.920,NaN,Has relevent experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0.0


## B. Khám phá dữ liệu (thường đan xen với pha tiền xử lý dữ liệu)

### 1.Mỗi dòng có ý nghĩa gì? Có vấn đề với các dòng có ý nghĩa khác nhau không?

* Mỗi dòng là thông tin của từng ứng viên, trong đó mỗi ô của dòng là dữ liệu tương ứng với cột của ô đó
* Không có vấn đề các dòng có ý nghĩa khác nhau

### 1.1 Kiểm tra các dòng có bị trùng lặp hay không?

In [4]:
have_duplicated_rows = np.any(df.duplicated())
have_duplicated_rows

False

Các dòng không bị trùng lặp.

### 2.Mỗi cột có ý nghĩa gì?
            
            - enrollee_id : ID với từng ứng viên

            - city: Mã thành phố

            - city_ development _index : Chỉ số phát triển của thành phố (theo tỷ lệ)

            - gender: Giới tính

            - relevent_experience: Kinh nghiệm liên quan của ứng viên

            - enrolled_university: Loại khóa học đại học đã đăng ký nếu có

            - education_level: Trình độ học vấn

            - major_discipline: Ngành học chính

            - experience: Kinh nghiệm (tính theo năm)

            - company_size: Số lượng nhân viên trong công ty của người sử dụng lao động hiện tại

            - company_type : Loại chủ lao động hiện tại

            - last_new_job: Sự khác biệt về số năm giữa công việc trước đây và công việc hiện tại

            - training_hours: Thời gian hoàn thành huấn luyện

            - target: 0 – Không nhảy việc, 1 – Nhảy việc

### 3.Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

In [5]:
df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                    float64
dtype: object

In [6]:
df['experience'].unique()

array(['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4',
       '10', '14', '18', '19', '12', '3', '6', '9', '8', '20', nan],
      dtype=object)

Vì trong feature experience có các dữ liệu như <1, 1, 2, ..., 20, >20 nên ta sẽ chuyển dữ liệu sang dạng các khoảng như <1, 1-5, 6-10, 11-15, 16-20, >20

In [7]:
df['experience'].replace({'1':'1-5',
                            '2':'1-5',
                            '3':'1-5',
                            '4':'1-5',
                            '5':'1-5',
                            '6':'6-10',
                            '7':'6-10',
                            '8':'6-10',
                            '9':'6-10',
                            '10':'6-10',
                            '11':'11-15',
                            '12':'11-15',
                            '13':'11-15',
                            '14':'11-15',
                            '15':'11-15',
                            '16':'16-20',
                            '17':'16-20',
                            '18':'16-20',
                            '19':'16-20',
                            '20':'16-20'},inplace=True)

### 4.Với mỗi cột, các giá trị được phân bố như thế nào?

### 4.1 Với các cột có giá trị dạng số

**Các cột dạng số:** _'enrollee_id', 'city_development_index', 'training_hours', 'target'_ 

DataFrame `summary_df` bên dưới chứa thông tin phân bố giá trị của các cột trên:
- Tên của các cột là tên của các cột số trong `df`
- Tên của các dòng có ý nghĩa là: 
 + "missing_ratio": tỉ lệ phần trăm các giá trị thiếu
 + "count": số lượng giá trị 
 + "mean": giá trị trung bình
 + "std": độ lệch chuẩn
 + "min": giá trị nhỏ nhất
 + "25%": giá trị phân vị 25%
 + "50%": giá trị phân vị 50%
 + "75%": giá trị phân vị 75%
 + "max": giá trị lớn nhất

In [8]:
summary_df = df.describe().round(1)
missing_ratio = pd.DataFrame(df[['enrollee_id', 'city_development_index', 'training_hours', 'target']].isna().sum() * 100/ len(df)).T.round(1)
missing_ratio = missing_ratio.set_axis(['missing_ratio'])
summary_df = pd.concat([missing_ratio, summary_df])
summary_df

,enrollee_id,city_development_index,training_hours,target
missing_ratio,0.0,0.0,0.0,0.0
count,19158.0,19158.0,19158.0,19158.0
mean,16875.4,0.8,65.4,0.2
std,9616.3,0.1,60.1,0.4
min,1.0,0.4,1.0,0.0
25%,8554.2,0.7,23.0,0.0
50%,16982.5,0.9,47.0,0.0
75%,25169.8,0.9,88.0,0.0
max,33380.0,0.9,336.0,1.0


Theo quan sát ở trên, ta thấy các cột có giá trị số không có giá trị thiếu. Mọi thứ trong có vẻ ổn.

### 4.2 Với các cột có giá trị categorical

Các giá trị sẽ tính:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu 
- Số lượng các giá trị (các giá trị ở đây là các giá trị khác nhau và ta không xét giá trị thiếu)
- Tỉ lệ % (từ 0 đến 100) của mỗi giá trị được sort theo tỉ lệ % giảm dần (ta không xét giá trị thiếu, tỉ lệ là tỉ lệ so với số lượng các giá trị không thiếu)

In [9]:
def missing_ratio(x):
    return (x.isna().mean()*100)
def num_values(x):
    return len(x.dropna().value_counts())
def value_ratios(x):
    count_value =  x.dropna().value_counts().sort_values(ascending=False)
    return (count_value/count_value.sum()*100).round(1).to_dict()
cat_df_info = df.select_dtypes(include="object").agg(func=[missing_ratio, num_values, value_ratios])
cat_df_info

,city,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
missing_ratio,0.0,23.53064,0.0,2.014824,2.401086,14.683161,0.339284,30.994885,32.049274,2.207955
num_values,123,3,2,3,5,6,6,8,6,6
value_ratios,"{'city_103': 22.7, 'city_21': 14.1, 'city_16':...","{'Male': 90.2, 'Female': 8.5, 'Other': 1.3}","{'Has relevent experience': 72.0, 'No relevent...","{'no_enrollment': 73.6, 'Full time course': 20...","{'Graduate': 62.0, 'Masters': 23.3, 'High Scho...","{'STEM': 88.7, 'Humanities': 4.1, 'Other': 2.3...","{'1-5': 30.7, '6-10': 26.2, '>20': 17.2, '11-1...","{'50-99': 23.3, '100-500': 19.4, '10000+': 15....","{'Pvt Ltd': 75.4, 'Funded Startup': 7.7, 'Publ...","{'1': 42.9, '>4': 17.6, '2': 15.5, 'never': 13..."


Các cột gender, company_size, company_type đều có số lượng giá trị thiếu khá lớn, trên 22%. Các số liệu khác trong có vẻ ổn

### 5. Tiền xử lí dữ liệu

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

Số lượng giá trị bị thiếu.

In [11]:
df.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
dtype: int64

Ta thấy có khá nhiều giá trị bị thiếu và đa số đều là categorical. Vì vậy, ta sẽ xóa các dòng bị thiếu.

In [13]:
df.dropna(inplace=True)
df.shape

(8955, 14)

Số lượng dòng bị thiếu khá nhiều, khoảng 1/2 của dữ liệu.

## C. Đưa ra các câu hỏi có ý nghĩa cần trả lời

### 1. Số lượng Data Scientists trên 10 năm kinh nghiệm đang tìm kiếm công việc mới trong Top 5 thành phố phát triển nhất?

Lợi ích khi trả lời được câu hỏi:
- Với vai trò là doanh nghiệp sẽ biết thêm thông tin để thay đổi chính sách đãi ngộ hay điều chỉnh môi trường làm việc để giữ chân các nhân viên tài năng của minh.
- Liệu các Data Scientists nhiều năm kinh nghiệm có nhu cầu tìm kiếm thách thức mới cho bản thân mình.
- Thị trường việc làm về Data Scientist ở trong Top 5 thành phố phát triển nhất.

## D. Tiền xử lý và phân tích dữ liệu để trả lời cho từng câu hỏi

### 1. Số lượng Data Scientists trên 10 năm kinh nghiệm đang tìm kiếm công việc mới trong Top 5 thành phố phát triển nhất?

## E. Tổng hợp lại quá trình thực hiện đồ án